In [1]:
from algosdk import account, mnemonic, logic
from algosdk.future import transaction
from algosdk.v2client import algod
from pyteal import compileTeal, Mode, Approve

ALGOD_TOKEN = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"
TESTNET_ALGOD_RPC = "https://testnet-api.algonode.network"

algod_client = algod.AlgodClient(ALGOD_TOKEN, TESTNET_ALGOD_RPC)
sp_func = algod_client.suggested_params
on_complete_param = transaction.OnComplete.NoOpOC
    
mnemonic_1 = open("../wallet_1").read().replace(',', ' ')
mnemonic_2 = open("../wallet_2").read().replace(',', ' ')
mnemonic_3 = open("../wallet_3").read().replace(',', ' ')

alice_private_key = mnemonic.to_private_key(mnemonic_1)
alice_address = account.address_from_private_key(alice_private_key)
bob_private_key = mnemonic.to_private_key(mnemonic_2)
bob_address = account.address_from_private_key(bob_private_key)
carol_private_key = mnemonic.to_private_key(mnemonic_3)
carol_address = account.address_from_private_key(carol_private_key)

print("Alice {} Balance: {}".format(alice_address, 
                algod_client.account_info(alice_address).get('amount') / 1e6))
print("Bob   {} Balance: {}".format(bob_address, 
                algod_client.account_info(bob_address).get('amount') / 1e6))
print("Carol {} Balance: {}".format(carol_address, 
                algod_client.account_info(carol_address).get('amount') / 1e6))

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response

Alice GZ4IJXXNRFT23E6SLUOSSUWN2LUDFQTX4F6SXF5EP27LFWTOWHPFANLYIQ Balance: 28.157993
Bob   CIK3P7U4PZBJESWQ3XFDCZDXEZ3JZLWY6XZHY4A4KEJCA5SYYIPBT6W7Y4 Balance: 15.382008
Carol K7ZJP3J7SYYNB42DPZMRY56X32HVTYOOZRA2ASE2IND4LSQKCL3CE2Z2YU Balance: 8.998


## 1. Asset Creation and OptIn

In [2]:
# Creation
create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDC", asset_name="Meson Test Circle USD"
))
usdc_index = create_asset_tx['asset-index']
print("Asset [Meson Test Circle USD] created success! Asset id: %s\n" % usdc_index)

create_asset_tx = submit_transaction(alice_private_key, transaction.AssetCreateTxn(
    alice_address, sp_func(), int(1e18), 6, False, unit_name="mUSDT", asset_name="Meson Test Tether"
))
usdt_index = create_asset_tx['asset-index']
print("Asset [Meson Test Tether] created success! Asset id: %s\n" % usdt_index)

Signed transaction with txID: HDWVCTCRDGV2HKW3CMSHIU7PWHAU4JLRNKUYUMSCNFDULIBJWAJQ
Confirmed on round 27782688!
Asset [Meson Test Circle USD] created success! Asset id: 159625952

Signed transaction with txID: 4RCBMTPRPBIBHJFTIOK3FBNV3O45QKRCXJ5ZLZQQQJH7225ZJXKQ
Confirmed on round 27782690!
Asset [Meson Test Tether] created success! Asset id: 159625960



In [3]:
# Optin
submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdc_index
)); print("Bob opt in mUSDC asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdc_index
)); print("Carol opt in mUSDC asset success!\n")

submit_transaction(bob_private_key, transaction.AssetOptInTxn(
    bob_address, sp_func(), usdt_index
)); print("Bob opt in mUSDT asset success!\n")

submit_transaction(carol_private_key, transaction.AssetOptInTxn(
    carol_address, sp_func(), usdt_index
)); print("Carol opt in mUSDT asset success!\n")

Signed transaction with txID: KCJ2VLNWXE6DMKCWBJGMCE7X6RCKH4PD36A5OUWYYGTIXHXAPPYA
Confirmed on round 27782694!
Bob opt in mUSDC asset success!

Signed transaction with txID: SLUQ4P2SUI6PM2LKHMRJRR7PZZSMQT4UVACD7VOMHM3X3PSAQQZA
Confirmed on round 27782696!
Carol opt in mUSDC asset success!

Signed transaction with txID: 7BG5CZQSV6FP5WCBL5RF2VIWRDQIDUHI4I3BDQ6DBPFCBTMUHU7Q
Confirmed on round 27782698!
Bob opt in mUSDT asset success!

Signed transaction with txID: 6PNU4UX3GM3ML5VAVE7MDNIVDJ42MKUBWZ7TZ2OWNFDZIMYUWN2Q
Confirmed on round 27782700!
Carol opt in mUSDT asset success!



In [4]:
# Transfer
asset_amount = 2e12

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdc_index
)); print("Bob received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdc_index
)); print("Carol received %d mUSDC!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), bob_address, int(asset_amount), usdt_index
)); print("Bob received %d mUSDT!" % (asset_amount / 1e6))

submit_transaction(alice_private_key, transaction.AssetTransferTxn(
    alice_address, sp_func(), carol_address, int(asset_amount), usdt_index
)); print("Carol received %d mUSDT!" % (asset_amount / 1e6))

Signed transaction with txID: KWMUCYPTF64VAEJAZNQKUBKSIEPJNRRKZAEZ57Z5ASAW6D52K35A
Confirmed on round 27782727!
Bob received 2000000 mUSDC!
Signed transaction with txID: CUIH6K6UFA6WNHVZL5TR2EX2XTS4EFVV3CS7VTQXKHBDQZ5OOUGQ
Confirmed on round 27782729!
Carol received 2000000 mUSDC!
Signed transaction with txID: ZQ3FHOFPHDURLD27Q54XCJPL6BKYDNCL72GCQDMSYY43FH23TLUQ
Confirmed on round 27782731!
Bob received 2000000 mUSDT!
Signed transaction with txID: FJ56KIGN5BOUHDMW6H7J5QIHFV3P6QX6CKGEST4W2ALDVFEUSWKQ
Confirmed on round 27782733!
Carol received 2000000 mUSDT!


## 2. Create Contract

In [2]:
import base64
import sys
sys.path.append('../contract')
from typing import List

In [3]:
usdc_index = 27782688
usdt_index = 27782690

def submit_transaction(private_key: str, unsigned_txn: transaction.Transaction):
    signed_txn = unsigned_txn.sign(private_key)
    txid = algod_client.send_transaction(signed_txn)
    print("Signed transaction with txID: {}".format(txid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, txid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(txid)
    return transaction_response
    
def submit_transaction_group(private_key: str, unsigned_txns: List[transaction.Transaction]):
    gid = transaction.calculate_group_id(unsigned_txns)
    signed_txns = []
    for unsigned in unsigned_txns:
        unsigned.group = gid
        signed = unsigned.sign(private_key)
        signed_txns.append(signed)
    gtxid = algod_client.send_transactions(signed_txns)
    print("Signed transaction group with gtxID: {}".format(gtxid))
    confirmed_txn = transaction.wait_for_confirmation(algod_client, gtxid, 3)
    print("Confirmed on round {}!".format(confirmed_txn['confirmed-round']))
    transaction_response = algod_client.pending_transaction_info(gtxid)
    return transaction_response

def compile_program(client, source_code):
    compile_response = client.compile(source_code)
    return base64.b64decode(compile_response['result'])

blank_program = compile_program(algod_client, compileTeal(
    Approve(), Mode.Application, version=8
))

### 2.1 MesonSwap

In [4]:
# Creation
from MesonSwap import mesonSwapMainFunc

meson_swap = compile_program(algod_client, teal_sentences := compileTeal(
    mesonSwapMainFunc(), Mode.Application, version=8
))
create_app_tx = submit_transaction(alice_private_key, transaction.ApplicationCreateTxn(
    alice_address, sp_func(), on_complete_param, meson_swap, blank_program,
    transaction.StateSchema(10, 0), transaction.StateSchema(5, 0)
))
print("Create MesonSwap success! App id: %s, App Address: %s" % (
    mesonswap_index := create_app_tx['application-index'],
    mesonswap_address := logic.get_application_address(mesonswap_index)
))

Signed transaction with txID: EMYKAWMXDNBGK4ZSKBYVYIFLIP2WU63UHD3IE3KZGXZH6ODIGMMA
Confirmed on round 27801886!
Create MesonSwap success! App id: 159695273, App Address: D44YFTAO2MIFRUAYPHCRAAH5RG2WJREOIJBGRRRAGH4PEEO7SMNHNLRKXA


In [ ]:
# Optin itself
submit_transaction(alice_private_key, transaction.ApplicationOptInTxn(
    alice_address, sp_func(), mesonswap_index
))
print("MesonSwap Optin itself success! Can add local variable now.")

In [6]:
# Add USDC and USDT
submit_transaction(alice_private_key, transaction.ApplicationCallTxn(
    alice_address, sp_func(), mesonswap_index, on_complete_param,
    app_args=['addSupportToken', usdc_index, 1]
))

AlgodHTTPError: TransactionPool.Remember: transaction N4GIDQJZJ6NCASOWE2KEE2CCGPLFD3ZYELTJ67HJLNVRP5FM74RA: logic eval error: invalid Account reference for mutation D44YFTAO2MIFRUAYPHCRAAH5RG2WJREOIJBGRRRAGH4PEEO7SMNHNLRKXA. Details: pc=767, opcodes=concat
intc_1 // 0
app_local_put
